# **Folium Visualization**
- https://cosmosproject2015.tistory.com/220
- **[지도 위 데이터 표시하기](https://rk1993.tistory.com/entry/Python%EC%A7%80%EB%8F%84-%EB%8D%B0%EC%9D%B4%ED%84%B0-%EC%8B%9C%EA%B0%81%ED%99%94-Folium-%EA%B8%B0%EC%B4%88-%EC%8B%A4%EC%8A%B5%ED%95%98%EA%B8%B0)**
- **[Folium Tutorial](https://python-visualization.github.io/folium/quickstart.html#Markers)**
- **[Module 설치하기](https://somjang.tistory.com/entry/Python-Folium-%ED%95%9C%EA%B8%80-%EA%B9%A8%EC%A7%90-%ED%98%84%EC%83%81-%ED%95%B4%EA%B2%B0%ED%95%98%EA%B8%B0)**
```r
! pip install folium
! pip install git+https://github.com/python-visualization/branca.git@master
```

## **1 Loading the table**
엑셀 원본데이터 불러오기

In [1]:
import pandas as pd
table = pd.read_csv('./data/artstore_match_v7.csv')
table.tail(2)

,group,artist_no,artist,index,fix,view,title,type,ceo,phone,address,addressfull,addressroad,lon,lat
143,C,12,임우현,C-12-3,0,1,임서하헤어마벨,미용업,임서하,010-8689-8650,장군봉1길 29,서울특별시 관악구 장군봉1길 29,서울특별시 관악구 봉천동 930-3,126.941729,37.481842
144,C,12,임우현,C-12-4,0,1,정가,음식점,유영옥,010-5331-7715,남부순환로 1710,서울특별시 관악구 남부순환로 1710 이너스빌딩,서울특별시 관악구 봉천동 945-10 이너스빌딩,126.939874,37.482753


In [2]:
table[table["artist"]=='이은비']

,group,artist_no,artist,index,fix,view,title,type,ceo,phone,address,addressfull,addressroad,lon,lat
67,B,5,이은비,B-5-2,0,1,완산포차,주점,나경엽,010-8707-8269,신림로65길 31,서울특별시 관악구 신림로65길 31,서울특별시 관악구 신림동 1433-71,126.927691,37.484744
68,B,5,이은비,B-5-3,0,1,코코멍냥,기타판매,김미선,010-3065-8226,신림동7길 8,서울특별시 관악구 신림동7길 8,서울특별시 관악구 신림동 1459-4,126.926053,37.486684
69,B,5,이은비,B-5-4,0,1,신림 큐슈라멘&스시,음식점,하석진,010-5342-5029,"신림동7길 6, 지하",서울특별시 관악구 신림동7길 6,서울특별시 관악구 신림동 1459-3,126.925976,37.486784


In [3]:
from collections import Counter
for _ in ["A","B","C"]:
    table_master = table[table['group']==_]
    check_list = Counter(table_master['artist_no'].to_list())
    print(dict(check_list))

{1: 4, 2: 4, 3: 4, 4: 4, 5: 4, 6: 4, 7: 4, 8: 4, 9: 4, 10: 4, 11: 4, 12: 4, 13: 4}
{1: 3, 2: 4, 3: 4, 4: 4, 5: 3, 6: 4, 7: 4, 8: 3, 9: 4, 10: 4, 11: 4, 12: 4}
{1: 4, 2: 4, 3: 4, 4: 4, 5: 4, 6: 4, 7: 4, 8: 4, 9: 4, 10: 4, 11: 4, 12: 4}


## **2 Additional DataSet**
2020년 11월 24일 데이터 추가하기

In [4]:
# from data.arterior import Arterior
# _table = [
#     ["D",1,"미정","D-1-1",1,1,"장수우렁불백","식당","이규엽","010-2326-1063","관악로148 2층"],
#     ["D",1,"미정","D-1-2",1,1,"뚱띵이왕갈비","식당","김영희","010-9744-9428","서림동94-258"],
# ]
# table_add = pd.DataFrame(_table)
# table_add.columns = list(table.columns[:-4])
# table_add = Arterior().naverAddressTableInsert(table_add,'address',11)
# table = pd.concat([table, table_add]).reset_index(drop=True)
# table.to_csv('./data/artstore_match_v6.csv', index=None)
# table.tail(2)

In [5]:
# table['lon'] = list(map(lambda x : float(x), table['lon']))
# table['lat'] = list(map(lambda x : float(x), table['lat']))

# table['lat'].mean()
# table['lon'].mean()

## **3 Folium Visualization**
Folium 시각화 작업
    # 'white', 'beige', 'pink', 'lightgray','lightgreen', 'lightblue','lightred',  
    # color_map = ['gray','red', 'purple', 'lightred',  'black',  'green',  'darkblue', 
    #    'blue', 'darkgreen',  'darkpurple', 'darkred', 'orange', 'cadetblue'] * 10

In [6]:
import folium, re
from folium import plugins
from folium.features import DivIcon

def folium_map(table, title='atrist', zoom=14):    
    """rCircle Rounded Map Marker"""
    map_data = folium.Map([table['lat'].mean(), table['lon'].mean()], zoom_start=zoom)
    table = table.reset_index(drop=True)
    color_master_map = {"A":"darkblue", "B":"orange", "C":"darkred", "D":"green"}
    color_map = ['lightgray','lightgreen', 'lightblue',
                 'lightred','lightred','cadetblue','white'] * 30    
    # 개별 데이터 Spot 찍기
    for _ in range(len(table)):
        _table = table.iloc[_, :] # pandas.Series 추출하기
        color_text = color_map[_table['artist_no']]
        index_name = _table['index']
        title, manager = index_name[2:], _table["group"]
        font_size = 15
        if index_name.find("D") != -1:
            font_size = 25
        # popUp Message 내용 정의하기
        popup_text = f"""{title}<br/>
            {_table['title']} ({_table['type']}) , 
            {_table['ceo']} : {_table['phone']}<br/>
            {_table['address']} | {_table['addressroad']}"""

        # Text Div ICON 내용 정의하기
        marker_sytle = DivIcon(
            icon_size = (150,50),
            icon_anchor = (7,20), # {color_text}
            html = f'''<div style=
                "font-size: {font_size}px;
                 color: black;
                 text-shadow: 2px 2px 2px {color_text};
                 font-weight: bold;
                 margin-top: 20px;
                 margin-left: 15px">{title}</div>''')

        # Marker Icon Style 정의하기
        icon_style = plugins.BeautifyIcon(
            number = manager, # + title,
            icon_shape   = "circle", 
            inner_icon_style = f'font-size:{font_size-2}px',
            border_color = color_master_map[_table["group"]], 
            text_color = 'black', # color_text, 
            border_width = 4)

        folium.Marker( # Marker & Popup Text
            [ _table['lat'], _table['lon']], # tooltip = popup_text,
            icon = icon_style).add_to(map_data)     

        map_data.add_child(folium.Marker( # Tooltip Text
            [_table['lat'], _table['lon']],
            tooltip = popup_text,
            icon = marker_sytle,
        ))
    return map_data

map_data = folium_map(table, title='artist', zoom=14)
map_data.save('./web/gkart_v5.html')
map_data

## **4 Table for Django DataBases**
WEB Portfolio 에 활용하는 테이블 생성하기

In [7]:
import pandas as pd
table = pd.read_csv('./data/artstore_match_v6.csv')
table.tail(2)

,group,artist_no,artist,index,fix,view,title,type,ceo,phone,address,addressfull,addressroad,lon,lat
144,C,12,임우현,C-12-3,0,1,임서하헤어마벨,미용업,임서하,010-8689-8650,장군봉1길 29,서울특별시 관악구 장군봉1길 29,서울특별시 관악구 봉천동 930-3,126.941729,37.481842
145,C,12,임우현,C-12-4,0,1,정가,음식점,유영옥,010-5331-7715,남부순환로 1710,서울특별시 관악구 남부순환로 1710 이너스빌딩,서울특별시 관악구 봉천동 945-10 이너스빌딩,126.939874,37.482753


In [10]:
col_filter = ['title','artist','type','ceo','phone','address','addressfull','addressroad','lon','lat']

In [12]:
table.loc[:, col_filter].to_csv("./data/storeDjango.csv", index=None, header=False)

In [9]:
list(table.columns)

['group',
 'artist_no',
 'artist',
 'index',
 'fix',
 'view',
 'title',
 'type',
 'ceo',
 'phone',
 'address',
 'addressfull',
 'addressroad',
 'lon',
 'lat']